In [1]:
from types import SimpleNamespace
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import numpy as np
import pandas as pd
import json, os, pysam, math

# paths
path_workdir = Path("/home/b05b01002/HDD/project_nanoprep_re")
path_output = Path("./output")
os.makedirs(path_output, exist_ok=True)

In [2]:
# NanoPrePro/Pychopper full-length read file name template
path_bam = lambda wildcards: \
    path_workdir / f"outputs/Minimap2/minimap2_genome/{wildcards.software}/{wildcards.beta_or_backend}/{wildcards.name}_{wildcards.accuracy}.bam"

# wildcards
names_accuracies = {
    "egr-109-bio1": ["sup", "hac", "fast"],
    "egr-109-bio2": ["sup", "hac", "fast"],
    "lch-109-bio1": ["sup", "hac", "fast"],
    "lch-109-bio2": ["sup", "hac", "fast"],
    "ptr-109-bio1": ["sup", "hac", "fast"],
    "ptr-109-bio2": ["sup", "hac", "fast"],
    "ptr-111-bio1": ["sup", "hac", "fast"],
    "ont-10x-human": ["sup", "hac", "fast"],
    "ont-visium-mouse": ["sup", "hac", "fast"],
    "mouse-retina-subset1": ["pre-called"],
    "mouse-retina-subset2": ["pre-called"]
}
accuracies = [
    "sup",
    "hac",
    "fast",
    "pre-called"
]
betas = [f"beta{i}" for i in ["0.1", "0.2", "0.3", "0.4", "0.5", "0.6", "0.7", "0.8", "0.9", "1.0", "1.5", "2.0"]]
backends = ["phmm", "edlib"]
softwares = ["pychopper", "nanoprep", "raw"]
pretty_names = {
    "pychopper": "Pychopper",
    "nanoprep": "NanoPreP"
}

Get soft clips

In [3]:
def soft_clips(entry):
    return entry.query_alignment_start + (entry.query_length - entry.query_alignment_end)

In [4]:
data = pd.DataFrame()
for name, accuracies in names_accuracies.items():
    for accuracy in accuracies:
        for software in softwares:
            # pychopper do not work with PCS114
            if (accuracy == "pre-called") & (software == "pychopper"):
                continue
            # 
            beta_or_backends = {
                "nanoprep": betas,
                "pychopper": backends,
                "raw": ["raw"]
            }[software]
            clean_name = {
                "pychopper": lambda x: x.split("|")[-1],
                "nanoprep": lambda x: x,
                "raw": lambda x: x
            }[software] # check this
            for beta_or_backend in beta_or_backends:
                wildcards = SimpleNamespace(
                    software=software,
                    beta_or_backend=beta_or_backend,
                    name=name,
                    accuracy=accuracy
                )
                fname = path_bam(wildcards)
                with pysam.AlignmentFile(fname, "rb") as handle:
                    name_softclip = [[clean_name(entry.query_name), soft_clips(entry)] for entry in handle]
                data_tmp = pd.DataFrame(name_softclip)
                data_tmp.columns = ["read_name", "soft_clipped_bases"]
                data_tmp["software"] = software
                data_tmp["beta_or_backend"] = beta_or_backend
                data_tmp["name"] = name
                data_tmp["accuracy"] = accuracy if not (name in ["mouse-retina-subset1", "mouse-retina-subset2"]) else "sup"
                data = pd.concat([data, data_tmp])

In [5]:
data.to_csv("soft_clip.csv", index=False)

Plot

In [6]:
data = pd.read_csv("soft_clip.csv")
data.head()

,read_name,soft_clipped_bases,software,beta_or_backend,name,accuracy
0,000fd2b2-5f4c-48ec-a6ee-d5ebc151a08c,2,pychopper,phmm,egr-109-bio1,sup
1,00224db5-37ce-4471-8eca-ca2576e5c520,0,pychopper,phmm,egr-109-bio1,sup
2,00065f53-4fe3-4140-96c7-baf5ee451eca,15,pychopper,phmm,egr-109-bio1,sup
3,001dff81-6494-4319-a2e8-3b11677c9791,30,pychopper,phmm,egr-109-bio1,sup
4,0028c6fd-21e3-4f50-9964-457460b2f91b,9,pychopper,phmm,egr-109-bio1,sup


In [ ]:
palette = {}
palette["phmm"], palette["edlib"] = sns.color_palette(["#8a6d53", "#ab7849"], 2)
for i, j in zip(betas, sns.color_palette("viridis_r", len(betas))):
    palette[i] = j
palette["raw"] = "gray"
col_wrap = 4
col = "name"
col_values = data[col].unique()
x="beta_or_backend"
y="soft_clipped_bases"
hue = "beta_or_backend"
subplot_aspect = 1.4
subplot_height = 4
subplot_width = subplot_height * subplot_aspect
fig_rows = math.ceil(len(col_values) / col_wrap)
fig_cols = col_wrap

# plot
fig, axes = plt.subplots(fig_rows, col_wrap, figsize=(subplot_width * fig_cols, subplot_height * fig_rows))
plt.subplots_adjust(wspace=0.1, hspace=0.1)
for name, ax in zip(col_values, axes.flatten()):
    sns.boxplot(
        data[data["name"] == name],
        x=x,
        y=y,
        hue=hue,
        palette=palette,
        legend=True,
        showfliers=False,
        ax=ax
    )
    ax.set_xlabel("")
    ax.set_ylabel("")
    ax.set_title(name)

# adjustment
show_legend = (0, fig_cols - 1)
for n_row in range(axes.shape[0]):
    for n_col in range(axes.shape[1]):
        ax = axes[n_row][n_col]
        if (n_row + 1) * (n_col + 1) > len(col_values):
            ax.remove()
            continue
        if (n_row, n_col) == show_legend:
            ax.legend(loc="center right", bbox_to_anchor=(1.5, 0.5))
        else:
            ax.legend_.remove()
        if n_row != (axes.shape[0] - 1):
            ax.set_xticklabels([])
        else:
            ax.tick_params(axis="x", rotation=70)

# save
fig.savefig("output/soft_clipped_bases.svg")